In [16]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip install googlenews
# !pip install gnews==0.3.6
# !pip install plotly==5.18.0
# ! pip install nbformat==5.9.2
# !pip install openai==1.3.7
# !pip install pyaml==23.9.7
# !pip show 

In [17]:
import sys
sys.path.append('..')

import os
import json
import pandas as pd

from tabulate import tabulate
from IPython.display import display, Markdown
# TEST

from datetime import datetime, timedelta
import pytz

from deltalake import DeltaTable
from deltalake.writer import write_deltalake

from openai import OpenAI

import yaml

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [18]:
with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)

query = config['query']
days_lookback = config['days_lookback']
max_results = config['max_results']

config

{'query': 'Eli Lilly Ozempic Risks', 'days_lookback': 90, 'max_results': 1000}

In [19]:
# Params
table_path_in = '../data/articles-scored'
table_path_out = '../data/articles-responded'
# --------------------------------------------

# query = "Ozempic"
# days_lookback = 30

current_datetime = datetime.now().astimezone(pytz.utc)
current_date = current_datetime.date()
start_date = (current_datetime - timedelta(days=days_lookback)).date()

# --------------------------------------------

start_date_str = str(start_date.strftime("%Y-%m-%d"))
current_date_str = str(current_date.strftime("%Y-%m-%d"))

print(
    f"""
    end_date: {current_date}
    start_date: {start_date}
    
    
    days_lookback: {days_lookback}
    """
)



    end_date: 2023-12-13
    start_date: 2023-09-14
    
    
    days_lookback: 90
    


In [20]:
# Query queries that have been ingested
DeltaTable(table_path_in).to_pandas(columns=['query']).value_counts()

query                  
Eli Lilly Ozempic Risks    70
Name: count, dtype: int64

In [21]:
df = DeltaTable(table_path_in).to_pandas(
    filters=[
        ('query','=',query),
        ('published date str','>', start_date_str),
    ]
)

df = df.loc[df['article text'].notna()] # can remove articles where scraping failed

df.head(1)

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords,sentiment_prompt,sentiment_score,sentiment_explanation,negative_claims,__index_level_0__
0,Intestinal Blockage Added as Potential Ozempic...,Intestinal Blockage Added as Potential Ozempic...,2023-10-04,https://news.google.com/rss/articles/CBMiUmh0d...,"{'href': 'https://www.drugwatch.com', 'title':...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-10-04 07:00:00,2023-10-04,The U.S. Food and Drug Administration has adde...,<!DOCTYPE html>\n<!--[if lt IE 8]> <html ...,Other side effects of Ozempic already listed o...,"[blockage, effects, drug, fda, intestinal, weg...",\n - Your task is to read the article below...,9,The sentiment score is high due to the numerou...,- Intestinal blockage (ileus) added as a poten...,76


In [22]:
df = df.sort_values(by=['sentiment_score'], ascending=False)
df.head(2)

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,article html,article summary,article keywords,sentiment_prompt,sentiment_score,sentiment_explanation,negative_claims,__index_level_0__
0,Intestinal Blockage Added as Potential Ozempic...,Intestinal Blockage Added as Potential Ozempic...,2023-10-04,https://news.google.com/rss/articles/CBMiUmh0d...,"{'href': 'https://www.drugwatch.com', 'title':...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-10-04 07:00:00,2023-10-04,The U.S. Food and Drug Administration has adde...,<!DOCTYPE html>\n<!--[if lt IE 8]> <html ...,Other side effects of Ozempic already listed o...,"[blockage, effects, drug, fda, intestinal, weg...",\n - Your task is to read the article below...,9,The sentiment score is high due to the numerou...,- Intestinal blockage (ileus) added as a poten...,76
12,Florida pharmacies selling GLP-1 knockoffs fac...,Florida pharmacies selling GLP-1 knockoffs fac...,2023-11-30,https://news.google.com/rss/articles/CBMif2h0d...,"{'href': 'https://www.fiercepharma.com', 'titl...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-30 08:00:00,2023-11-30,Seven weeks after a federal judge dismissed (P...,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre...",Seven weeks after a federal judge dismissed (P...,"[drugs, versions, lawsuits, companies, knockof...",\n - Your task is to read the article below...,9,The sentiment score is given a 9 because the a...,- Novo Nordisk alleges that Brooksville Pharma...,52


## process for prompting openai with article info, with traceability

- Each row in the table is an article. Functions can be used to iterate through the table and add additional column.
- For each prompt, we want to retain the prompt in the table, as well as the message history, and response.


In [23]:
client = OpenAI()

def print_df(df):
    return tabulate(df, headers='keys', tablefmt='html')

def generate_openai_response_json(prompt, model="gpt-3.5-turbo-1106"):
    """
    Generic function to generate a GPT JSON response
    """
    response = client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt}
    ]
    )
    json_response = response.choices[0].message.content
    return json.loads(json_response)


def generate_openai_response(prompt, model="gpt-3.5-turbo-1106"):
    """
    Generic function to generate a GPT STRING response
    """
    response = client.chat.completions.create(
    model=model,
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    )
    return response.choices[0].text



def generate_prompt(query, title, article_text, published_date_str, sentiment_score, sentiment_explanation, negative_claims):
    prompt = f"""
    Task:
    Your task is to review the below article, and its sentiment analysis, to build a case against the article's claims:
    - Provide specific counterpoints to refute the claims of the article.
    - Provide a 'disputability' score, from 0-10, where 10 is most disputable.
    - Provide a list of search queries for additional research you would need to enhance your case.

    Inputs:
    - Search term used to find this article: {query}

    - The article:
    -   Title: {title}
        - Published Date: {published_date_str}
        - Article: {article_text}
    
    - Sentiment Analysis:
        - Sentiment score: {sentiment_score}
        - Sentiment explanation: 
            {sentiment_explanation}
        - Negtive claims: 
            {negative_claims}
    
    ## Outputs
    - Return the output as a JSON object with the following: 
        - 'counter_points': (string with bullet points delimited by new lines), 
        - 'disputability_score': (integer as string from 0-10)
        - 'research_queries': (list of strings)
    
    """
    return prompt


def generate_response(prompt):
    response = generate_openai_response_json(prompt)
    
    return response['counter_points'], response['disputability_score'], response['research_queries']


def apply_response(df, column_names = ['counter_points', 'disputability_score', 'research_queries']):

    df_copy = df.copy()

    new_columns = df_copy.apply(lambda row: generate_response(row['rebuttal_prompt']), axis=1 , result_type='expand')
    new_columns.columns = column_names

    for col in new_columns.columns:
        df_copy.loc[:, col] = new_columns[col].copy()

    return df_copy



In [24]:
i=0
article_text = df.iloc[i]['article text']
published_date_str = df.iloc[i]['published date str']
title = df.iloc[i]['title']
query = df.iloc[i]['query']
sentiment_score = df.iloc[i]['sentiment_score']
sentiment_explanation = df.iloc[i]['sentiment_explanation']
negative_claims = df.iloc[i]['negative_claims']

prompt = generate_prompt(query, title, article_text, published_date_str, sentiment_score, sentiment_explanation, negative_claims)
display(Markdown(prompt))


    Task:
    Your task is to review the below article, and its sentiment analysis, to build a case against the article's claims:
    - Provide specific counterpoints to refute the claims of the article.
    - Provide a 'disputability' score, from 0-10, where 10 is most disputable.
    - Provide a list of search queries for additional research you would need to enhance your case.

    Inputs:
    - Search term used to find this article: Eli Lilly Ozempic Risks

    - The article:
    -   Title: Intestinal Blockage Added as Potential Ozempic Side Effect - DrugWatch.com
        - Published Date: 2023-10-04
        - Article: The U.S. Food and Drug Administration has added ileus, or blocked intestines, to the list of possible adverse reactions to the diabetes medication Ozempic. Other side effects of Ozempic already listed on the label are anaphylaxis, angioedema, rash, urticaria, cholecystitis and cholecystectomy.

Ozempic and similar medications Wegovy and Mounjaro belong to a family of drugs known as GLP-1 agonists. The active ingredient in Ozempic and Wegovy is semaglutide, while Mounjaro contains tirzepatide. Wegovy and Mounjaro already have a warning on their labels about ileus.

Pharmaceutical company Novo Nordisk told CNN in July that GLP-1 agonists have been used for many years and have gone through extensive studies.

“Gastrointestinal (GI) events are well-known side effects of the GLP-1 class,” Novo Nordisk said in a statement. “For semaglutide, the majority of GI side effects are mild to moderate in severity and of short duration. GLP-1’s are known to cause a delay in gastric emptying, as noted in the label of each of our GLP-1 RA medications. Symptoms of delayed gastric emptying, nausea and vomiting are listed as side effects.”

Along with a list of adverse reactions, Ozempic’s label includes this statement: “Because these reactions are reported voluntarily from a population of uncertain size, it is not always possible to reliably estimate their frequency or establish a causal relationship to drug exposure.”

See if You Qualify for a Lawsuit Our Partners Our Trusted Legal Partners Drugwatch partners with trusted law firms to help you take legal action. After submitting the form, one of Drugwatch's partners will contact you for a free case review.

Lawsuits Filed

This labeling change by the FDA comes just weeks after a Louisiana woman filed an Ozempic lawsuit and a Mounjaro lawsuit against drugmakers Novo Nordisk and Eli Lilly. In her suit, Jaclyn Bjorklund said while the manufacturers warned users about other side effects caused by Ozempic they failed to mention the risk of severe gastroparesis.

“Although there is some overlap in the terms, ileus and gastroparesis, they are not synonymous,” FDA spokesperson Chanapa Tantibanchachai said in a statement.

Labels for Ozempic and similar drugs already note that they cause “delay of gastric emptying,” Tantibanchachai said, but the regulator is continuing to monitor “reports of gastroparesis and other related terms” in real-world use of the drug.

“If newly identified safety signals are identified, the FDA will determine what, if any, actions are appropriate after a thorough review of available data,” she said.

Ozempic has recently seen a surge in demand because it may also help people lose weight. According to clinical trials, some of the most common side effects of Ozempic are abdominal pain, constipation, diarrhea, nausea and vomiting. The majority of these side effects should only last a few days or a few weeks. Some of the more serious side effects of Ozempic include:

Acute gallbladder disease

Acute kidney injury

Allergic reactions

Diabetic retinopathy

Hypoglycemia (when used with insulin or sulfonylurea)

Increased risk of thyroid tumors

Pancreatitis

Long-term use of Ozempic may result in chronic side effects, especially gastrointestinal issues. Some people are forced to stop taking the drug because the side effects don’t improve. This could lead to undereating or malabsorption of nutrients.

Ongoing Investigation Into Ozempic

The European Medicines Agency started investigating Ozempic and Wegovy in July after more than 100 reports claiming the drug could cause suicidal thoughts and self-harm. Novo Nordisk released a statement saying “no causal association” between self-harming thoughts and the drugs was found during its own safety monitoring checks.

Wegovy and Saxenda, another medication used to treat diabetes, also have an FDA warning about possible suicidal thoughts. The FDA Adverse Event Reporting System Public Dashboard has at least 60 reports since 2018 of suicidal ideation from people taking semaglutide, the active ingredient in Ozemic and Wegovy.

Popularity for the drug has skyrocketed, with many people using Ozempic for weight loss. Quarterly prescriptions for the drug and similar medications increased by 300% between 2020 and 2022, according to analytics firm Trilliant Health. The drug’s high demand has even caused a global shortage, which is expected to continue well into 2024.
    
    - Sentiment Analysis:
        - Sentiment score: 9
        - Sentiment explanation: 
            The sentiment score is high due to the numerous negative side effects and warnings associated with Ozempic, including the recent addition of intestinal blockage as a potential side effect by the FDA. Additionally, the surge in demand for the drug and the ongoing investigations contribute to the overall negative sentiment.
        - Negtive claims: 
            - Intestinal blockage (ileus) added as a potential adverse reaction to Ozempic
- Severe gastroparesis risk not mentioned in warnings
- Surge in demand for Ozempic causing global shortage
- Reports of suicidal thoughts and self-harm associated with the drug
- Adverse side effects such as acute gallbladder disease, acute kidney injury, allergic reactions, diabetic retinopathy, hypoglycemia, increased risk of thyroid tumors, and pancreatitis
    
    ## Outputs
    - Return the output as a JSON object with the following: 
        - 'counter_points': (string with bullet points delimited by new lines), 
        - 'disputability_score': (integer as string from 0-10)
        - 'research_queries': (list of strings)
    
    

In [25]:
prompt

"\n    Task:\n    Your task is to review the below article, and its sentiment analysis, to build a case against the article's claims:\n    - Provide specific counterpoints to refute the claims of the article.\n    - Provide a 'disputability' score, from 0-10, where 10 is most disputable.\n    - Provide a list of search queries for additional research you would need to enhance your case.\n\n    Inputs:\n    - Search term used to find this article: Eli Lilly Ozempic Risks\n\n    - The article:\n    -   Title: Intestinal Blockage Added as Potential Ozempic Side Effect - DrugWatch.com\n        - Published Date: 2023-10-04\n        - Article: The U.S. Food and Drug Administration has added ileus, or blocked intestines, to the list of possible adverse reactions to the diabetes medication Ozempic. Other side effects of Ozempic already listed on the label are anaphylaxis, angioedema, rash, urticaria, cholecystitis and cholecystectomy.\n\nOzempic and similar medications Wegovy and Mounjaro belo

In [26]:
counter_points, disputability_score, research_queries = generate_response(prompt)

output = f""" 

Negative claims: 
    \n{negative_claims}

Counter points: 
    \n{counter_points}

Disputability score: {disputability_score}

Research queries: {research_queries}

"""

display(Markdown(output))

 

Negative claims: 
    
- Intestinal blockage (ileus) added as a potential adverse reaction to Ozempic
- Severe gastroparesis risk not mentioned in warnings
- Surge in demand for Ozempic causing global shortage
- Reports of suicidal thoughts and self-harm associated with the drug
- Adverse side effects such as acute gallbladder disease, acute kidney injury, allergic reactions, diabetic retinopathy, hypoglycemia, increased risk of thyroid tumors, and pancreatitis

Counter points: 
    
1. The pharmaceutical company, Novo Nordisk, states that gastrointestinal events are well-known side effects of GLP-1 class and that the majority of these side effects are mild to moderate and of short duration. 2. The FDA continuously monitors safety signals and will take appropriate actions based on available data. 3. The reports of suicidal thoughts and self-harm associated with the drug have been investigated by Novo Nordisk and no causal association was found. 4. The surge in demand and global shortage may be attributed to the drug's potential for weight loss, which could be a positive indication for its use.

Disputability score: 7

Research queries: ['Long-term studies on the safety and efficacy of Ozempic and similar GLP-1 agonists', 'Comparison of reported adverse effects between Ozempic and other diabetes medications', "FDA's monitoring and assessment process for labeling changes and side effect reports", 'Clinical trials and real-world data on the efficacy and safety of Ozempic for weight loss']



In [27]:

df['rebuttal_prompt'] = df.apply(lambda row: generate_prompt(row['query'], row['title'], row['article text'], row['published date str'], row['query'], row['sentiment_explanation'], row['negative_claims'] ), axis=1)


In [28]:
df = df.sort_values(by=['sentiment_score'], ascending=False)
df = df.head(50)

In [30]:
# df

In [31]:

df = apply_response(df)


In [32]:
df = df.sort_values(by=['disputability_score'], ascending=False)

In [37]:
df.columns

Index(['title', 'description', 'published date', 'url', 'publisher', 'query',
       'query end date', 'published datetime', 'published date str',
       'article text', 'article html', 'article summary', 'article keywords',
       'sentiment_prompt', 'sentiment_score', 'sentiment_explanation',
       'negative_claims', '__index_level_0__', 'rebuttal_prompt',
       'counter_points', 'disputability_score', 'research_queries'],
      dtype='object')

In [40]:
df

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article text,...,article keywords,sentiment_prompt,sentiment_score,sentiment_explanation,negative_claims,__index_level_0__,rebuttal_prompt,counter_points,disputability_score,research_queries
41,Diabetes drug Mounjaro helped dieters shed 60 ...,Diabetes drug Mounjaro helped dieters shed 60 ...,2023-10-15,https://news.google.com/rss/articles/CBMihQFod...,"{'href': 'https://www.nbcchicago.com', 'title'...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-10-15 07:00:00,2023-10-15,The medicine in the diabetes drug Mounjaro hel...,...,"[drug, shed, weight, finds, dieters, helped, t...",\n - Your task is to read the article below...,7,The sentiment score of 7 is given because the ...,"- Side effects including nausea, diarrhea, and...",20,\n Task:\n Your task is to review the be...,"The study was funded by Eli Lilly, the manufac...",9,[Long-term safety and efficacy of tirzepatide ...
61,Mounjaro vs. Insulin: Which Is More Effective ...,Mounjaro vs. Insulin: Which Is More Effective ...,2023-10-03,https://news.google.com/rss/articles/CBMid2h0d...,"{'href': 'https://www.healthline.com', 'title'...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-10-03 07:00:00,2023-10-03,Share on Pinterest Mounjaro appears to be a sa...,...,"[trial, group, fastacting, weight, insulin, vs...",\n - Your task is to read the article below...,8,The sentiment negativity score is 8 because th...,- Mounjaro showed significantly fewer instanc...,73,\n Task:\n Your task is to review the be...,"- The trial was sponsored by Eli Lilly, the ma...",9,[Long-term cardiovascular safety of Mounjaro v...
54,US FDA approves Jardiance® for the treatment o...,US FDA approves Jardiance® for the treatment o...,2023-09-22,https://news.google.com/rss/articles/CBMieGh0d...,"{'href': 'https://investor.lilly.com', 'title'...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-09-22 07:00:00,2023-09-22,Approval adds to the treatment options for the...,...,"[disease, adults, risk, fda, approves, blood, ...",\n - Your task is to read the article below...,8,The sentiment score is given an 8 out of 10 du...,- Jardiance is not recommended for use to impr...,32,\n Task:\n Your task is to review the be...,While the article presents Jardiance as a new ...,9,"[Jardiance safety profile, Comparison of Jardi..."
21,Ozempic and other weight-loss drugs will power...,Ozempic and other weight-loss drugs will power...,2023-11-04,https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-04 07:00:00,2023-11-04,Weight-loss drugs could fuel huge gains for th...,...,"[gains, drugs, power, weight, ozempic, billion...",\n - Your task is to read the article below...,8,The sentiment score is given as 8 due to the f...,- The article lacks critical discussion about ...,51,\n Task:\n Your task is to review the be...,- The article lacks critical discussion about ...,9,"[Eli Lilly Ozempic safety concerns, Side effec..."
24,Should We End Obesity? | TIME - TIME,Should We End Obesity? | TIME TIME,2023-11-08,https://news.google.com/rss/articles/CBMiOWh0d...,"{'href': 'https://time.com', 'title': 'TIME'}",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-08 08:00:00,2023-11-08,It’s unusual for a medication to become a hous...,...,"[disease, health, weight, ozempic, end, patien...",\n - Your task is to read the article below...,8,The sentiment negativity score of 8 is given d...,- The widespread use and marketing of Ozempic ...,16,\n Task:\n Your task is to review the be...,- Ozempic and similar weight-loss medications ...,9,[Long-term side effects of Ozempic and other w...
12,Florida pharmacies selling GLP-1 knockoffs fac...,Florida pharmacies selling GLP-1 knockoffs fac...,2023-11-30,https://news.google.com/rss/articles/CBMif2h0d...,"{'href': 'https://www.fiercepharma.com', 'titl..."

In [41]:
write_deltalake(
    table_path_out,
    df.reset_index(),
    partition_by=['query', 'published date str'],
    mode='overwrite',
    # overwrite_schema=True,
    partition_filters=[
        ('query', '=', query),
        ('published date str', '>=', start_date_str), 
        ('published date str', '<=', current_date_str),
    ]
)

In [42]:
DeltaTable(table_path_out).to_pandas()

,index,title,description,published date,url,publisher,query,query end date,published datetime,published date str,...,article keywords,sentiment_prompt,sentiment_score,sentiment_explanation,negative_claims,__index_level_0__,rebuttal_prompt,counter_points,disputability_score,research_queries
0,44,Weight-loss drugs are impacting everything fro...,Weight-loss drugs are impacting everything fro...,2023-11-13,https://news.google.com/rss/articles/CBMiZ2h0d...,"{'href': 'https://fortune.com', 'title': 'Fort...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-13 08:00:00,2023-11-13,...,"[drugs, impacting, surgery, weight, impact, sn...",\n - Your task is to read the article below...,8,The sentiment score is 8 because the article d...,- Retailers and food/beverage companies are co...,12,\n Task:\n Your task is to review the be...,1. Although the article suggests that weight-l...,8,[Long-term effects of weight-loss drugs on con...
1,43,How does the weight loss drug Zepbound compare...,How does the weight loss drug Zepbound compare...,2023-11-13,https://news.google.com/rss/articles/CBMiXWh0d...,"{'href': 'https://www.uchealth.org', 'title': ...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-13 08:00:00,2023-11-13,...,"[drug, drugs, weight, ozempic, zepbound, work,...",\n - Your task is to read the article below...,8,The sentiment score is given as 8 due to the n...,- Zepbound and other weight loss drugs have hi...,11,\n Task:\n Your task is to review the be...,1. Zepbound and other weight loss drugs may no...,8,[Long-term efficacy and sustainability of weig...
2,36,Behold the Ozempic effect on business - Financ...,Behold the Ozempic effect on business Financi...,2023-11-19,https://news.google.com/rss/articles/CBMiP2h0d...,"{'href': 'https://www.ft.com', 'title': 'Finan...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-19 08:00:00,2023-11-19,...,"[reasons, reliable, ozempic, secure, ft, socia...",\n - Your task is to read the article below...,8,The sentiment score is given as 8 due to the a...,- The article highlights potential health risk...,9,\n Task:\n Your task is to review the be...,Counterpoints to refute the claims of the arti...,8,"[Ozempic clinical trial results, Eli Lilly reg..."
3,22,Eli Lilly says weight loss drug Zepbound is no...,Eli Lilly says weight loss drug Zepbound is no...,2023-12-06,https://news.google.com/rss/articles/CBMiZ2h0d...,"{'href': 'https://www.wxyz.com', 'title': 'WXY...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-12-06 20:42:50,2023-12-06,...,"[drug, weight, eli, doses, zepbound, youre, pa...",\n - Your task is to read the article below...,9,The sentiment score is 9 due to the numerous p...,"- Zepbound may cause tumors in the thyroid, in...",43,\n Task:\n Your task is to review the be...,1. The potential serious side effects reported...,8,[Long-term safety and efficacy of tirzepatide ...
4,41,Diabetes drug Mounjaro helped dieters shed 60 ...,Diabetes drug Mounjaro helped dieters shed 60 ...,2023-10-15,https://news.google.com/rss/articles/CBMihQFod...,"{'href': 'https://www.nbcchicago.com', 'title'...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-10-15 07:00:00,2023-10-15,...,"[drug, shed, weight, finds, dieters, helped, t...",\n - Your task is to read the article below...,7,The sentiment score of 7 is given because the ...,"- Side effects including nausea, diarrhea, and...",20,\n Task:\n Your task is to review the be...,"The study was funded by Eli Lilly, the manufac...",9,[Long-term safety and efficacy of tirzepatide ...
5,16,Ozempic Pills for Weight Loss Don't Reduce Pri...,Ozempic Pills for Weight Loss Don't Reduce Pri...,2023-11-17,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228,2023-11-17 08:00:00,2023-11-17,...,"[drugs, effects, sold, issues, versions, table...",\n - Your task is to read the article below...,9,The sentiment score is 9 due